# #0

In [14]:
# ---- ¿Que año, mes y # de iteraciones quieres correr? ----
anyo = '2020'
mes = 'Abril'
iters = 10


In [29]:
# ---- Librerías ---- 
import pandas as pd 
import numpy as np
import scipy
import csv 
import warnings
import unicodedata
warnings.filterwarnings('ignore')

In [41]:
#Obtener la distancia promedio de un grupo de m puntos
def get_avg_distance(vectors):    
    distance = scipy.spatial.distance.pdist(vectors, metric='euclidean')
    return distance

'''Test
v = np.array(
[[ 115, 241, 314,56],
[ 153, 413, 144,67],
[ 535, 2986, 41445,5656]])

get_avg_distance(v)
'''

'Test\nv = np.array(\n[[ 115, 241, 314,56],\n[ 153, 413, 144,67],\n[ 535, 2986, 41445,5656]])\n\nget_avg_distance(v)\n'

In [18]:
# ---- DB Apellidos Paternos ---- 
df_paternos = pd.read_excel('data publica/apellidos_paternos.xlsx', index_col=0, usecols = 'A:D',engine="openpyxl")

In [20]:
# ---- DB Employees ---- 
df_whole = pd.read_csv('data publica/unidos/'+ mes + anyo + '.csv', index_col=0)


In [21]:
## ---------- Remover tildes de los apellidos y pasar a mayuscula ---------- ##
cols = df_paternos.select_dtypes(include=[np.object]).columns
df_paternos[cols] =df_paternos[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
cols = df_whole.select_dtypes(include=[np.object]).columns
df_whole[cols] =df_whole[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
df_whole['Paterno'] = df_whole['Paterno'].str.upper()
df_whole['Materno'] = df_whole['Materno'].str.upper()

In [22]:
## ---------- Apellidos Paternos x Región ---------- ##
pd.options.display.float_format = '{:,.2f}'.format 
df_paternos['AP PATERNO'] = df_paternos['AP PATERNO'].str.upper()
df_paternos_region = df_paternos[df_paternos['AP PATERNO'].notna()]
df_paternos_region = df_paternos_region.groupby(['REG','AP PATERNO']).agg({'CANTIDAD': "sum"}).sort_values(by=['CANTIDAD'], ascending=False)
df_paternos_region = df_paternos_region[df_paternos_region['CANTIDAD'] > 1]
df_paternos_region['CANTIDAD'] = df_paternos_region['CANTIDAD'].astype(int)
## ---------- Apellidos Paternos en Chile ---------- ##
df_paternos_region = df_paternos_region.reset_index(level=['REG'])
df_paternos_total = df_paternos_region.groupby(['AP PATERNO']).agg({'CANTIDAD': "sum"}).sort_values(by=['CANTIDAD'], ascending=False)
df_paternos_total['PORCENTAJE'] = (df_paternos_total['CANTIDAD']/df_paternos_total['CANTIDAD'].sum())
df_paternos_total['CANTIDAD'] = df_paternos_total['CANTIDAD'].astype(int)
df_paternos_total = df_paternos_total.dropna()
df_paternos_total = df_paternos_total[df_paternos_total['CANTIDAD'] > 1]

In [23]:
df_paternos_total

,CANTIDAD,PORCENTAJE
AP PATERNO,,
GONZALEZ,444542,0.02
MUNOZ,349675,0.02
ROJAS,249660,0.01
DIAZ,246901,0.01
PEREZ,196733,0.01
...,...,...
CANOBA,2,0.00
CANNY,2,0.00
CANNIZZO,2,0.00


In [42]:
# Método MonteCarlo 
def MonteCarlo(population, sample_number, weights_column, iterations):
    
    avg_distance = []
    
    for i in range(0, iterations): #numero de iteraciones
        
        #esto saca un random example de la poblacion (general), le damos un "N" y un peso específico a cada apellido
        random_weighted_sample = population.sample(n = sample_number, replace = True, 
                                                   weights= weights_column, axis = 0)
        
        #transformar apellidos en matriz de NxM
        #TBD_function()
        #Tests
        
        v = np.array(
        [[ 115, 241, 314,56],
        [ 153, 413, 144,67],
        [ 535, 2986, 41445,5656]])

        #sacar la distancia promedio de este iteración
        avg_distance_i = get_avg_distance(v)
        avg_distance.append(avg_distance_i)
    
    return avg_distance

# Simulación para cada organismo

In [36]:
#Loop para cada organismo

dict_organismos = {}
organismos = set(df_whole['organismo_codigo'])

for organismo in organismos:
    
    #Se filtra los empleados por organismo
    df_organismo = df_whole[df_whole['organismo_codigo'] == organismo]    

    #Número de personas en el organismo
    total = len(df_organismo['Paterno'])    
    
    #Sacamos la region
    region = df_organismo['region']
    regiones = set(region)
    lenght = len(regiones)

    #Composición de una región
    if lenght == 1:
        regiones = list(val for val in regiones)
        region = regiones[0]
        region = unicodedata.normalize("NFKD",region).encode("ascii","ignore").decode("ascii")
        code_region = region_codes[region]  
        population = df_paternos_region[df_paternos_region['REG'] == code_region] 

    #Composición toda la poblacion
    else:
        population = df_paternos_total
        population['PORCENTAJE'] = population['CANTIDAD']/population['CANTIDAD'].sum()
        output = MonteCarlo(population, total, 'PORCENTAJE', iters)
        
        dict_organismos[organismo] = output

In [39]:
dict_organismos

{'AM006': [array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.25210846, 41758.54134186])],
 'AW004': [array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.25210846, 41758.54134186]),
  array([  245.0489747 , 41603.252108